<a href="https://colab.research.google.com/github/DayalStrub/ecir2021tutorial/blob/main/other/2-simplest-thing-Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q python-terrier
!pip install -q git+https://github.com/terrier-org/pyterrier.git

In [2]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize

In [3]:
import pyterrier as pt
pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


PyTerrier 0.5.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [4]:
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')

In [5]:
next(iter(dataset.get_corpus_iter()))

OrderedDict([('title',
              'Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia'),
             ('doi', '10.1186/1471-2334-1-6'),
             ('date', '2001-07-04'),
             ('abstract',
              'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common

In [6]:
N = 10_000

pt_index_path = './index_cord19'

# create the index, using the IterDictIndexer indexer 
indexer = pt.index.IterDictIndexer(pt_index_path, overwrite=True) # , blocks=True

# we give the dataset get_corpus_iter() directly to the indexer
# while specifying the fields to index and the metadata to record
index_ref = indexer.index(
    (datum for i, datum in enumerate(dataset.get_corpus_iter()) if i < N),
    fields=('abstract',), # TODO should this be a list?
    # meta={'docno' : 26, 'text' : 2048, 'abstract' : 2048}
    meta=['docno','abstract'],
    )

index = pt.IndexFactory.of(index_ref)

07:48:19.455 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed

07:48:32.573 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Indexed 2143 empty documents


In [7]:
print(index.getCollectionStatistics().toString())

Number of documents: 10000
Number of terms: 36191
Number of postings: 673758
Number of fields: 1
Number of tokens: 1040094
Field names: [abstract]
Positions:   false



In [11]:
queries = dataset.get_topics(variant='title').head(3)
queries.head()

[INFO] If you have a local copy of https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/0307a37b6b9f1a5f233340a769d538ea
[INFO] download error: HTTPSConnectionPool(host='ir.nist.gov', port=443): Max retries exceeded with url: /covidSubmit/data/topics-rnd5.xml (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f4f7684e110>: Failed to establish a new connection: [Errno 113] No route to host')). Retrying from start.
[INFO] download error: HTTPSConnectionPool(host='ir.nist.gov', port=443): Max retries exceeded with url: /covidSubmit/data/topics-rnd5.xml (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f4f768625d0>: Failed to establish a new connection: [Errno 113] No route to host')). Retrying from start.


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
OSError: [Errno 113] No route to host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 839, in _validate_conn
    conn.connect()
  File "/usr/local/lib/python3.7/dist-packages/urll

[WARNING] Download failed: HTTPSConnectionPool(host='ir.nist.gov', port=443): Max retries exceeded with url: /covidSubmit/data/topics-rnd5.xml (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f4f768628d0>: Failed to establish a new connection: [Errno 113] No route to host'))
[ERROR] Internal Python error in the inspect module.
Below is the traceback from this internal error.

[INFO] 
Unfortunately, your original traceback can not be constructed.



TypeError: ignored

In [ ]:
M = 1002 # TODO is 1000 the max ever returned? Probably bug

bm25 = pt.BatchRetrieve(index, wmodel="BM25", num_results=M)
rm3 = pt.rewrite.RM3(index)
bm25_text = pt.BatchRetrieve(index, wmodel="BM25", metadata=["docno", "abstract"], num_results=M)

rm3_pipe = bm25 >> rm3 >> bm25_text >> pt.apply.rename({'abstract':'text'})

In [ ]:
# TODO Question: in rm3_pipe, is final bm25 just reranking initial bm25 results, or does it re-consider all docs?

In [ ]:
df_part = (bm25 >> rm3).transform(queries)

In [ ]:
df_out = rm3_pipe.transform(queries)
df_out # .loc[df_out["qid"] == "1", :]

In [ ]:
df_bm = (bm25 >> bm25_text).transform(queries)

In [ ]:
# pd.assertequal()

In [ ]:
df_out.loc[:, ["qid", "docid"]].groupby("qid").count()

In [ ]:
def pr_linear_reranker(
    df,
    *,
    n_top = 10,
    n_bottom = 100, # TODO should actually get 0 score documents? how to do this in pyterrier?
    col_text = 'text'
):
  df_tmp = df # .copy()
  df_relevant = df_tmp.sort_values("rank").reset_index(drop=True).loc[0:n_top, ["docid", col_text]]
  df_relevant["label"] = 1

  df_not_relevant = df_tmp.sort_values("rank", ascending=False).reset_index(drop=True).loc[0:n_bottom, ["docid", col_text]]
  df_not_relevant["label"] = 0

  df_train = pd.concat([df_relevant, df_not_relevant])

  text_transformer = Pipeline([
      ('bow', CountVectorizer()),
      ('tfidf', TfidfTransformer()),
  ])

  preprocessor = ColumnTransformer(
      transformers=[
          # ('num', num_transformer, col_num),
          # ('cat', cat_transformer, col_cat),
          ('text', text_transformer, col_text)
      ]
  )

  crf = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', LogisticRegression())
  ])

  X_train = df_train.loc[:, ["docid", col_text]]
  y_train = df_train["label"]

  crf.fit(X_train, y_train);

  preds = crf.predict_proba(df_tmp.loc[:, ["docid", col_text]])

  df_tmp["score"] = preds[:, 1] # True class second?
  df_tmp.sort_values("score", ascending=False, inplace=True)
  df_tmp.reset_index(drop=True, inplace=True)
  df_tmp["rank"] = df_tmp.index
  return df_tmp

In [ ]:
def normalise_score(df, norm="max"):
  normalized_data = normalize(df["score"].to_numpy().reshape(1, -1), norm=norm)[0]
  df["score"] = normalized_data
  return df

In [ ]:
class SimpleTransformer(pt.TransformerBase):
  def transform(self, df, **kwargs):
    df_score = pr_linear_reranker(df)
    return normalise_score(df_score)

In [ ]:
df_eg = df_out.copy().loc[df_out["qid"] == "2", :].reset_index(drop=True)

pr_linear_reranker(df_eg)

In [ ]:
pr_pipe = rm3_pipe >> pt.apply.by_query(pr_linear_reranker) >> pt.apply.by_query(normalise_score)

df_out_2 = pr_pipe.transform(queries)
df_out_2.loc[df_out_2["qid"] == "2", :]

In [ ]:
# TODO how to run rm3_pipe only once? CHECK caching

alpha = 0.5

pipe = alpha * rm3_pipe + (1 - alpha) * pr_pipe

In [ ]:
# NOTE Experiment is indeed/actually a function
pt.Experiment(
    retr_systems=[bm25, rm3_pipe, pr_pipe, pipe],
    names=["BM25", "BM25-RM3-BM25", "BM25-RM3-BM25-PRlog", "simplest"],
    topics=queries,
    qrels=dataset.get_qrels(), # TODO figure out cord19 qrels
    eval_metrics=["map"]
              )